In [1]:
quietly {
use OPM_database_clean.dta, clear
}

In [2]:
set more off 

label define yncs 1 "Yes" 0 "No", replace

*Variable Creation*
tab V_PC_None, m
label define VCPENO 1 "VC" 2 "PE" 0 "None", replace
label var V_PC_None VCPENO
*Venture Capital Dummy*
gen VC_dum=.
	replace VC_dum=1 if V_PC_None==1
	replace VC_dum=0 if V_PC_None==2 | V_PC_None==0
label var VC_dum "Venture capital"
 label val VC_dum yncs
tab VC_dum V_PC_None, m
*Private Equity Dummy*
gen PE_dum=.
	replace PE_dum=1 if V_PC_None==2
	replace PE_dum=0 if V_PC_None==1 | V_PC_None==0
label var PE_dum "Private Equity"
 label val PE_dum yncs
tab PE_dum V_PC_None, m
*No Venture Capital/Private Equity Dummy*
gen NoVCPE_dum=.
	replace NoVCPE_dum=1 if V_PC_None==0
	replace NoVCPE_dum=0 if V_PC_None==1 | V_PC_None==2
label var NoVCPE_dum "No VC/PE"
 label val NoVCPE_dum yncs
tab NoVCPE_dum V_PC_None, m
*Venture Capital or Private Equity Dummy*
gen VC_or_PE_dum=.
	replace VC_or_PE_dum=1 if V_PC_None==1 | V_PC_None==2
	replace VC_or_PE_dum=0 if V_PC_None==0
label var VC_or_PE_dum "PE/VC"
 label val VC_or_PE_dum yncs
tab VC_or_PE_dum V_PC_None, m


order (Institution-Visible) Captivity_AR_INDEF
*Making sure any variables coded as 999 replaced to missing*
foreach var of varlist Start_mo_contract-Captivity_AR_INDEF {
	replace `var'=. if `var'==999
}

tab Captivity_AR_INDEF, m

*Creating variable to capture contracts with no rev share/no chg per head setup*
tab Rev_share_w_TP TP_chgs_per_head, m
gen no_rs_ph=.
	replace no_rs_ph=1 if Rev_share_w_TP==0 & TP_chgs_per_head==0
	replace no_rs_ph=0 if Rev_share_w_TP==1 & TP_chgs_per_head==0
	replace no_rs_ph=0 if Rev_share_w_TP==0 & TP_chgs_per_head==1
	label val no_rs_ph yncs
list Provider Institution if Rev_share_w_TP==0 & TP_chgs_per_head==0

*Visibility*
tab Visible, m
*Visible categorical*
gen Visiblerc=.
	replace Visiblerc=0 if Visible==0
	replace Visiblerc=1 if Visible==1
	replace Visiblerc=2 if Visible==2
	replace Visiblerc=3 if Visible==3
	replace Visiblerc=4 if Visible==4
tab Visible Visiblerc, m
*Visible category 1*
gen invisible_1=.
	replace invisible_1=1 if Visiblerc==0
	replace invisible_1=0 if Visiblerc==1 | Visiblerc==2 | Visiblerc==3 | ///
	Visiblerc==4
	label var invisible_1 "Invisibility"
 label val invisible_1 yncs
tab invisible_1 Visiblerc, m
*Visible category 1 & 2*
gen invisible_2=.
	replace invisible_2=1 if Visiblerc==0 | Visiblerc==1 
	replace invisible_2=0 if Visiblerc==2 | Visiblerc==3 | ///
	Visiblerc==4 
 label val invisible_2 yncs
tab invisible_2 Visiblerc, m

*Visible category 1 & 2 & 3*
gen invisible_3=.
	replace invisible_3=0 if Visiblerc==0 | Visiblerc==1 | Visiblerc==2 
	replace invisible_3=1 if  Visiblerc==3 | Visiblerc==4 
 label val invisible_3 yncs
tab invisible_3 Visiblerc, m
*Visible category 1 & 2 & 3 & 4* Most visible partnership = reference category*
gen invisible_4=.
	replace invisible_4=0 if Visiblerc==0 | Visiblerc==1 | Visiblerc==2 | Visiblerc==3
	replace invisible_4=1 if  Visiblerc==4 
 label val invisible_4 yncs
tab invisible_4 Visiblerc, m

*Creating Learning Management identifier*
*Providers identified in paper*
*Angel Learning or Blackboard, Desire2Learn (D2L), Instructure, Longsight, and Moodle.
encode Provider, gen(provider_rc)
	tab provider_rc, m
	tab provider_rc, m nol

gen LMS=0
	replace LMS=1 if provider_rc==8 | provider_rc==15 | ///
	provider_rc==29 | provider_rc==48 | provider_rc==49 | provider_rc==58 | ///
	provider_rc==62
	label val LMS yncs
tab LMS, m


tab Term_primary_contract, m

gen cat_termofcontract=.
replace cat_termofcontract=1 if Term_primary_contract<1
replace cat_termofcontract=2 if Term_primary_contract==1
replace cat_termofcontract=3 if Term_primary_contract>1 & Term_primary_contract<5
replace cat_termofcontract=4 if Term_primary_contract>=5 & Term_primary_contract<10
replace cat_termofcontract=5 if Term_primary_contract>=10
replace cat_termofcontract=6 if Term_primary_contract==888
replace cat_termofcontract=. if Term_primary_contract==999

label var cat_termofcontract "Term of contract categories"
label define categories_term 1 "Less than 1 year" 2 "1 year" 3 "Between 1 and 4 years" ///
4 "Between 5 and 9 years" 5 "10 years or more" 6 "Indefinite", replace
label val cat_termofcontract categories_term
tab Term_primary_contract cat_termofcontract, m
tab cat_termofcontract, m



*Reviewing descriptive statistics for all variables in paper*
order VC_dum PE_dum VC_or_PE_dum Rev_share_w_TP TP_chgs_per_head ///
TP_recruiting TP_course_dev TP_instruction Expansion Captivity invisible_1 ///
invisible_2 invisible_3 no_rs_ph LMS

	tab1 VC_dum-VC_or_PE_dum, m

	tab1 Rev_share_w_TP-no_rs_ph, m

	tab Rev_share_w_TP TP_chgs_per_head, m

foreach var of varlist (VC_dum-VC_or_PE_dum) {
	tab1 Rev_share_w_TP-invisible_2 if `var'==1
}


*Labeling for descriptives table*
label var Term_primary_contract "Term of the contract"
label var For_Profit "For Profit third-party"
label var LMS "Learning Management System"
label var NoVCPE_dum "Neither"
label var Rev_share_w_TP "Revenue Share"
label var TP_chgs_per_head "Pricing per head/credit hour"
label var no_rs_ph "Pricing per service"
label var TP_recruiting "Recruitment"
label var TP_course_dev "Course or program development"
label var TP_instruction "Instructional Services"
label var invisible_1 "(In)visibility"
label var Expansion "Expansion"
label var Captivity_AR_INDEF "Captivity"






  V_PC_None |      Freq.     Percent        Cum.
------------+-----------------------------------
          0 |         55       24.02       24.02
          1 |         62       27.07       51.09
          2 |        112       48.91      100.00
------------+-----------------------------------
      Total |        229      100.00



(229 missing values generated)

(62 real changes made)

(167 real changes made)




   Venture |              VCPENO
   capital |         0          1          2 |     Total
-----------+---------------------------------+----------
        No |        55          0        112 |       167 
       Yes |         0         62          0 |        62 
-----------+---------------------------------+----------
     Total |        55         62        112 |       229 

(229 missing values generated)

(112 real changes made)

(117 real changes made)




   Private |              VCPENO
    Equity |         0          1          2 |     Total
-----------+------------

     |                                                 Institution |
     |                        Central New Mexico Community College |
     +-------------------------------------------------------------+

     +-------------------------------------------------------------+
 22. |                                       Provider              |
     |                                     Blackboard              |
     |-------------------------------------------------------------|
     |                                                 Institution |
     |                                       Central Texas College |
     +-------------------------------------------------------------+

     +-------------------------------------------------------------+
 23. |                                       Provider              |
     |                                      Longsight              |
     |-------------------------------------------------------------|
     |                          

     |                                                 Institution |
     |                               Jacksonville State University |
     +-------------------------------------------------------------+

     +-------------------------------------------------------------+
 50. |                                       Provider              |
     |                                    Instructure              |
     |-------------------------------------------------------------|
     |                                                 Institution |
     |                            Johnson County Community College |
     +-------------------------------------------------------------+

     +-------------------------------------------------------------+
 52. |                                       Provider              |
     |                                     Blackboard              |
     |-------------------------------------------------------------|
     |                          

     |                                                 Institution |
     |                      New Mexico Higher Education Department |
     +-------------------------------------------------------------+

     +-------------------------------------------------------------+
 85. |                                       Provider              |
     |                              Aspect Consulting              |
     |-------------------------------------------------------------|
     |                                                 Institution |
     |                             New Mexico Highlands University |
     +-------------------------------------------------------------+

     +-------------------------------------------------------------+
 86. |                                       Provider              |
     |                                            D2L              |
     |-------------------------------------------------------------|
     |                          

     |                                                 Institution |
     |                                            San Juan College |
     +-------------------------------------------------------------+

     +-------------------------------------------------------------+
111. |                                       Provider              |
     |              Evaluation KIT (by Watermark LLC)              |
     |-------------------------------------------------------------|
     |                                                 Institution |
     |                                            San Juan College |
     +-------------------------------------------------------------+

     +-------------------------------------------------------------+
115. |                                       Provider              |
     |                                      JER Group              |
     |-------------------------------------------------------------|
     |                          

...

          1 |          9        3.93       57.21
          2 |         89       38.86       96.07
          3 |          3        1.31       97.38
          4 |          4        1.75       99.13
          . |          2        0.87      100.00
------------+-----------------------------------
      Total |        229      100.00

(229 missing values generated)

(30 real changes made)

(9 real changes made)

(89 real changes made)

(3 real changes made)

(4 real changes made)


           |                       Visiblerc
   Visible |         0          1          2          3          4 |     Total
-----------+-------------------------------------------------------+----------
      -999 |         0          0          0          0          0 |        91 
      -888 |         0          0          0          0          0 |         1 
         0 |        30          0          0          0          0 |        30 
         1 |         0          9          0          0          0 |       

                                 Ed2Go |         11        4.80       48.47
                                 Ed2go |          1        0.44       48.91
                              Edmentum |          1        0.44       49.34
                    Education Dynamics |          1        0.44       49.78
                  Elephant Productions |          1        0.44       50.22
                               Enspire |          1        0.44       50.66
     Evaluation KIT (by Watermark LLC) |          1        0.44       51.09
                                EverFi |          1        0.44       51.53
                            Everspring |          3        1.31       52.84
                      Exeter Education |          1        0.44       53.28
                       GetEducated.com |          1        0.44       53.71
                                 Guild |          1        0.44       54.15
           Harvard Business School HBX |          1        0.44       54.59
            

         63 |          1        0.44       73.80
         64 |          1        0.44       74.24
         65 |          3        1.31       75.55
         66 |          9        3.93       79.48
         67 |          6        2.62       82.10
         68 |          1        0.44       82.53
         69 |          2        0.87       83.41
         70 |          1        0.44       83.84
         71 |          1        0.44       84.28
         72 |          1        0.44       84.72
         73 |          1        0.44       85.15
         74 |          1        0.44       85.59
         75 |          1        0.44       86.03
         76 |          1        0.44       86.46
         77 |          1        0.44       86.90
         78 |          1        0.44       87.34
         79 |          1        0.44       87.77
         80 |          1        0.44       88.21
         81 |          1        0.44       88.65
         82 |          2        0.87       89.52
         83 |       

          0 |        145       63.32       63.32
          1 |         79       34.50       97.82
          . |          5        2.18      100.00
------------+-----------------------------------
      Total |        229      100.00

-> tabulation of TP_chgs_per_head  

TP_chgs_per |
      _head |      Freq.     Percent        Cum.
------------+-----------------------------------
          0 |        190       82.97       82.97
          1 |         34       14.85       97.82
          . |          5        2.18      100.00
------------+-----------------------------------
      Total |        229      100.00

-> tabulation of TP_recruiting  

TP_recruiti |
         ng |      Freq.     Percent        Cum.
------------+-----------------------------------
          0 |        157       68.56       68.56
          1 |         66       28.82       97.38
          . |          6        2.62      100.00
------------+-----------------------------------
      Total |        229      100.00

-> 

------------+-----------------------------------
          0 |         75       68.18       68.18
          1 |         35       31.82      100.00
------------+-----------------------------------
      Total |        110      100.00

-> tabulation of TP_course_dev if PE_dum==1 

TP_course_d |
         ev |      Freq.     Percent        Cum.
------------+-----------------------------------
          0 |         67       60.36       60.36
          1 |         44       39.64      100.00
------------+-----------------------------------
      Total |        111      100.00

-> tabulation of TP_instruction if PE_dum==1 

TP_instruct |
        ion |      Freq.     Percent        Cum.
------------+-----------------------------------
          0 |         85       77.27       77.27
          1 |         25       22.73      100.00
------------+-----------------------------------
      Total |        110      100.00

-> tabulation of Expansion if PE_dum==1 

  Expansion |      Freq.     Percent 

In [3]:
save "OPM_database_paperJN.dta", replace

file OPM_database_paperJN.dta saved
